##### AlexNet:
<img src="AlexNet/AlexNet.jpg">

In [19]:
import tensorflow as tf

def create_placeholder(x_shape, num_class, name = 'input_data'): #数据占位空间
    '''x_shape:输入图像维度（如[224,224,3]）
       num_class:输出类别数目（如10）
    '''
    with tf.name_scope(name):
        X = tf.placeholder(tf.float32, [None, x_shape[-3], x_shape[-2], x_shape[-1]], name = 'X')
        Y = tf.placeholder(tf.float32, [None, num_class], name = 'Y')
        tf.summary.image('X', X, 9) #加入tensorboard
    return X, Y

def conv(x, filter_h, filter_w, num_filters, stride_h, stride_x, name, padding = 'SAME', trainable = True): #卷积层操作
    '''x:输入（[N,H,W,C]）； filter_h, filter_w:卷积核维度； num_filters:卷积核个数；
       stride_h, stride_x:卷积滑动步长； padding:填充；name:名称
    '''
    input_channels = int(x.get_shape().as_list()[-1])
    with tf.variable_scope(name):
        W = tf.get_variable('weights', shape = [filter_h,filter_w,input_channels,num_filters], initializer = tf.contrib.layers.xaver_initializer(), trainable = trainable)
        b = tf.get_variable('biases', shape = [num_filters], initializer = tf.zeros_initializer(), trainable = trainable)
        conv = tf.nn.conv2d(x, W, strides=[1,stride_y,stride_x,1], padding = padding)
        relu = tf.nn.relu(tf.nn.bias_add(conv, b))
        tf.summary.histogram('weights', W)
        tf.summary.histogram('biases', b)
    return relu

def max_pool(x, filter_h, filter_w, stride_h, stride_x, name, padding = 'SAME'): #最大池化
    return tf.nn.max_pool(x, ksize = [1, filter_h, filter_w, 1], strides = [1, stride_h, stride_w, 1], padding = padding, name = name)
    
def fc(x, input_dim, output_dim, name, relu=True, trainable = True): #全连接层
    with tf.variable_scope(name):
        W = tf.get_variable('weights', shape = [input_dim, output_dim], initializer = tf.contrib.layers.xavier_initializer(), trainable=trainable)
        b = tf.get_variable('biases', shape = [output_dim], trainable=trainable)
        tf.summary.histogram('weights', W)
        tf.summary.histogram('biases', b)
        if relu:
            return tf.nn.relu(tf.matmal(x, W) + b)
        else:
            return tf.add(tf.matmul(x, W), b)

def lrn(x, radius, alpha, beta, name, bias=1.0): #局部响应归一化
    return tf.nn.local_response_normalization(x, depth_radius=radius, alpha=alpha, beta=beta, bias=bias, name=name)

def dropout(x, keep_prob, name): #dropout
    return tf.nn.dropout(x, keep_prob, name=name)

class AlexNet():
    def __init__(self, x, y, model_dir,
                 batch_size=1, learning_rate=0.01, 
                 keep_prob=0.5, num_steps=10000):
        self.TRAIB_IMAGES = x
        self.TRAIN_LABELS = y
        self.NUM_CLASS = y.shape[1]
        self.BATCH_SIZE = batch_size
        self.LEARNING_RATE = learning_rate
        self.KEEP_PROB = keep_prob
        self.NUM_STEPS = num_steps
        self.SAVE_PATH = model_dir
        self.X,self.Y = create_placeholder(self.TRAIB_IMAGES, self.NUM_CLASS)
#         self.backward()
        
    def forward(self):
        '''Create the network'''

        # 1st Layer: Conv(relu) -> Lrn -> Pool
        conv1 = conv(self.X, 11, 11, 96, 4, 4, padding='VALID', name='conv1')
        norm1 = lrn(conv1, 2, 1e-4, 0.75, name='norm1')
        pool1 = max_pool(norm1, 3, 3, 2, 2, padding='VALID', name='pool1')
        
        # 2nd Layer: Conv(relu) -> Lrn -> Pool
        conv2 = conv(pool1, 5, 5, 256, 1, 1, name='conv2')
        norm2 = lrn(conv2, 2, 1e-4, 0.75, name='norm2')
        pool2 = max_pool(norm2, 3, 3, 2, 2, padding='VALID', name='pool2')
        
        # 3rd Layer: Conv(relu)
        conv3 = conv(pool2, 3, 3, 384, 1, 1, name='conv3')
        
        # 4th Layer: Conv(relu)
        conv4 = conv(conv3, 3, 3, 384, 1, 1, name='conv4')
        
        # 5th Layer: Conv(relu) -> Pool
        conv5 = conv(conv4, 3, 3, 256, 1, 1, name='conv5')
        pool5 = max_pool(conv5, 3, 3, 2, 2, padding='VALID', name='pool5')
        
        # 6th Layer: Flatten -> FC(relu) -> Dropout
        flattened = tf.contrib.layers.flatten(pool5)
        fc6 = fc(flattened, flattened.get_shape().as_list()[-1], 4096, name='fc6')
        dropout6 = dropout(fc6, self.KEEP_PROB)
        
        # 7th Layer: FC(relu) -> Dropout
        fc7 = fc(dropout6, 4096, 4096, name='fc7')
        dropout7 = dropout(fc7, self.KEEP_PROB)
        
        # 7th Layer: FC(relu=False)
        fc8 = fc(fc7, 4096, self.NUM_CLASS, relu=False, name='fc8')
        
        return fc8
    
    def backward(self):
        num_epoch = 100
        num_batches = int(num_examples / self.BATCH_SIZE)
        
        Z = self.forward()
        global_step = tf.Variable(0, trainable=False, name='num_steps')
        tf.summary.scalar('learning_rate', self.LEARNING_RATE)
        with tf.name_scope('Optimizer'):
            loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=self.Z, labels=self.Y))
            train_step = tf.train.GradientDescentOptimizer(self.LEARNING_RATE).minimize(loss, global_step=global_step)
            tf.summary.scalar('loss', loss)
        with tf.name_scope('accuracy'):
            correct_prediction = tf.equal(tf.argmax(Z,1), tf.argmax(self.Y,1))
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name='accuracy')
            tf.summary.scalar('train', accuracy)
            
        config = tf.ConfigProto()
        config.gpu_options.allow_growth = True
        saver = tf.train.Saver()
        with tf.Session(config=config) as sess:
            merged = tf.summary.merge_all()
            writer = tf.summary.FileWriter(SAVE_PATH, sess.graph)
            sess.run(tf.global_variables_initializer())
            ckpt = tf.train.get_checkpoint_state(SAVE_PATH)
            if ckpt and ckpt.model_checkpoint_path:
                saver.restore(sess, ckpt.model_checkpoint_path)
                
            for epoch in range(num_epoch):
                for num_batch in range(num_batches):
                    _, loss_value, acc, summary, step = sess.run([train_step, loss, accuracy, merged, global_step], 
                                                                 feed_dict = {self.X:batch_images, self.Y:batch_labels})
                    writer.add_summary(summary, step)
                    print('INFO: After %d training iteration(s), loss is %.5f, batch_train accuracy is %.4f' % (step, loss_value, acc))
                    if step % 100 ==0:
                        saver.save(sess, self.SAVE_PATH+'AlexNet_Model.ckpt', global_step = global_step)
                        print('AlexNet_Model.ckpt-%d already saved!' % step)
                    if step >= NUM_STEPS: return
#                 saver.save(sess, self.SAVE_PATH+'AlexNet_Model.ckpt-%d'% (epoch+1)*num_batches)



In [ ]:
'''
train.py  (x, y, model_dir, batch_size=1, learning_rate=0.01, keep_prob=0.5, num_steps=10000)
Example usage:
    ./train.py --model_dir='./path/' \
        --num_steps=10000 \

'''
# import tensorflow as tf
# import os

# TRAIN_IMAGES = 
# TRAIN_LABELS = 
# BATCH_SIZE = 256
# # NUM_STEPS = 10000
# # MODEL_DIR

# flags = tf.app.flags
# flags.DEFINE_string('model_dir', None, 
#                     'Path to output model directory where event and checkpoint files will be written.')
# flags.DEFINE_integer('num_steps', None, 'Number of train steps.')

# FLAGS = flags.FLAGS
# MODEL_DIR = FLAGS.model_dir
# NUM_STEPS = FLAGS.num_steps
# if not os.path.exists(MODEL_DIR): os.mkdir(MODEL_DIR)

# model = AlexNet(TRAIN_IMAGES, TRAIN_LABELS, MODEL_DIR, BATCH_SIZE, NUM_STEPS)
# model.backward()

# if __name__ == '__main__':
#     tf.app.run()

In [18]:
'''
valuation.py  (x, y, model_dir, batch_size, learning_rate=0.01, keep_prob=0.5, num_steps)
'''
import tensorflow as tf

# IMAGES = 
# LABELS = 
# MODEL_DIR = 
model = AlexNet(IMAGES, LABELS, MODEL_DIR, keep_prob=1)
x = model.X
y = model.Y
y_ = model.forward()
saver = tf.train.Saver()
correct_prediction = tf.equal(tf.argmax(y_, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
    ckpt = tf.train.get_checkpoint_state(MODEL_DIR)
    if ckpt and ckpt.model_checkpoint_path:
        saver.restore(sess, ckpt.model_checkpoint_path)
        steps = ckpt.model_checkpoint_path.split('/')[-1].split('-')[-1]
        acc = sess.run(accuracy, feed_dict = {x:IMAGES, y:LABELS})
        print('After %s training step(s), evaluation accuracy is %.4f' % (steps, acc))
    else:
        print('No checkpoint file found!')
        return

NameError: name 'IMAGES' is not defined